In [ ]:
# Basic implementation of Baseball-Elimination algorithm
# Monotonic game setup of payoff pair (1,0) => Winner gets 1 point, loser gets 0 points at each game
import networkx as nx
from scipy import random
import numpy as np
import time
#lets check the running time
start_time = time.time()
############################### CHANGE THESE VARIABLES TO PLAY AROUND ###########################################
#Team 6 is selected for checking the elimination
id_of_our_team = 6
#Initially we are generating random data to work on 
number_of_teams = 10
number_of_games = 100
lost_games_upper_bound = 30
#################################################################################################################
#Creating matrix of remaining games in the league
A = random.randn(number_of_teams,number_of_teams)
B = np.dot(A,A.transpose())
Remaining_games_matrix = np.int_(B)
Remaining_games_matrix = abs(Remaining_games_matrix)
Remaining_games_matrix[range(number_of_teams), range(number_of_teams)] = 0
#print(Remaining_games_matrix)
#Finding out how many games each team has till the end of the season by summing columns for each row
remaining_games = np.sum(Remaining_games_matrix, axis=1)
#print(remaining_games)

In [ ]:
#creating loss games array randomly, from values between 1 and lost_games_upper_boundS
lost_games= np.random.randint(1,lost_games_upper_bound,number_of_teams)
#print(lost_games)
#Find number of wins in each team
won_games = number_of_games - (remaining_games + lost_games)
#print(won_games)
#at this point we have enough information for proceeding with baseball elimination algorithm

In [ ]:
# architecture of max-flow network for the algorithm
#  source -> set of teams pairs (layer 1) -> set of teams (layer 2) -> destination
# Source to layer1 edge capacities are number of games left between the pair vertex at layer 1
S_to_layer_1_edge_capacities = [None] * (number_of_teams * number_of_teams)
for i in range(number_of_teams):
   for j in range(number_of_teams):
      S_to_layer_1_edge_capacities[i*number_of_teams +j] = Remaining_games_matrix[i][j]
#print(S_to_layer_1_edge_capacities)
#second layer of edges are from each pair of teams to each team with infinite capacity (we just use number_of_teams^3, which is enough)
# therefore there are number_of_teams * number_of_teams * number_of_teams edges
layer_1_to_layer_2_edge_capacities = [None] * (number_of_teams * number_of_teams * number_of_teams)
for i in range(number_of_teams * number_of_teams * number_of_teams):
    layer_1_to_layer_2_edge_capacities[i] = number_of_teams * number_of_teams * number_of_teams
#capacity from layer2(teams) to sink depends on the team we want to solve the network for
#its the total number of win our team can get - current number of wins the team at the vertex has won

In [ ]:
#naming nodes for graph creation
#layer 1 nodes
layer_id = 'L1'
layer_1_ids = [None] * (number_of_teams * number_of_teams)
for i in range(0,len(S_to_layer_1_edge_capacities)):
    node_id = layer_id + str(i)
    layer_1_ids[i] = node_id
#print(layer_1_ids)
#layer 2 nodes
layer_id = 'L2'
layer_2_ids = [None] * (number_of_teams)
for i in range(0,number_of_teams):
    node_id = layer_id + str(i)
    layer_2_ids[i] = node_id
#print(layer_2_ids)

In [ ]:
#Creating network graph using networkx package
#initilizing graph nodes
graph = nx.DiGraph()
graph.add_node('S')
graph.add_nodes_from(layer_1_ids)
graph.add_nodes_from(layer_2_ids)
graph.add_node('T')
#assigning edge values for edges from S to layer 1 nodes
for i in range(0,len(layer_1_ids)):
    graph.add_edges_from([('S',layer_1_ids[i],{'capacity': S_to_layer_1_edge_capacities[i], 'flow': 0})])
#assigning edge values for edges from layer 1 to layer 2 nodes
for i in range(0,len(layer_1_ids)):
    for j in range(0,len(layer_2_ids)):
        graph.add_edges_from([(layer_1_ids[i],layer_2_ids[j],{'capacity': layer_1_to_layer_2_edge_capacities[i*number_of_teams+j], 'flow': 0})])   
#assigning edge values for edges from layer 2 to sink,but first we need to calculate edge values for our selected team
layer_2_to_destination_edge_capacities = [None] * number_of_teams
#calculating last layer of edge values
for j in range(number_of_teams):
    layer_2_to_destination_edge_capacities[j] = won_games[id_of_our_team] + remaining_games[id_of_our_team] - won_games[j]
#capacity of the edge for the selected team is the maximum amount of games it can win
layer_2_to_destination_edge_capacities[id_of_our_team] = won_games[id_of_our_team] + remaining_games[id_of_our_team]
#now assign it in the graph similarly
for i in range(0,number_of_teams):
    graph.add_edges_from([(layer_2_ids[i],'T',{'capacity': layer_2_to_destination_edge_capacities[i], 'flow': 0})])
#print(graph.edges)

In [ ]:
#Ford-Fulkerson for finding max flow
def ford_fulkerson(graph, source, sink):
    flow, path = 0, True
    
    while path:
        # search for path with flow reserve
        path, reserve = depth_first_search(graph, source, sink)
        flow += reserve

        # increase flow along the path
        for v, u in zip(path, path[1:]):
            if graph.has_edge(v, u):
                graph[v][u]['flow'] += reserve
            else:
                graph[u][v]['flow'] -= reserve
    
    the_Sum = sum(S_to_layer_1_edge_capacities)
    #print(flow)
    #print(the_Sum)
    if flow == the_Sum:
        print("the team with id %s, can strictly win or tie for championship" % (id_of_our_team))
    else:
        print("the team with id %s has no chance to win the league" % (id_of_our_team))

In [ ]:
#depth first search for finding augmented paths
def depth_first_search(graph, source, sink):
    undirected = graph.to_undirected()
    explored = {source}
    stack = [(source, 0, dict(undirected[source]))]
    
    while stack:
        v, _, neighbours = stack[-1]
        if v == sink:
            break
        
        # search the next neighbour
        while neighbours:
            u, e = neighbours.popitem()
            if u not in explored:
                break
        else:
            stack.pop()
            continue
        
        # current flow and capacity
        in_direction = graph.has_edge(v, u)
        capacity = e['capacity']
        flow = e['flow']
        neighbours = dict(undirected[u])

        # increase or redirect flow at the edge
        if in_direction and flow < capacity:
            stack.append((u, capacity - flow, neighbours))
            explored.add(u)
        elif not in_direction and flow:
            stack.append((u, flow, neighbours))
            explored.add(u)

    # (source, sink) path and its flow reserve
    reserve = min((f for _, f, _ in stack[1:]), default=0)
    path = [v for v, _, _ in stack]
    
    return path, reserve

In [ ]:
#theorem : there is a way for the team to tie or strictly win if g* is equal to  the max flow value
#where g* is the total number of games left to play
# Deciding on eliminaiton based on theorem stated above
ford_fulkerson(graph, 'S', 'T')
print("Execution time is --- %s seconds ---" % (time.time() - start_time))